In [ ]:
# default_exp lightningreapp

# Reappraisal Model
- Model management is handled by PyTorch Lightning. `LightningReapp` is a wrapper instance of a standard PyTorch neural network module such that the training/validation/testing logic is grouped with the logic of a forward pass. 
- Optimizer: [AdamW](https://pytorch.org/docs/stable/optim.html) 
- Loss Function: L2 Loss (Mean Squared Error)

In [ ]:
# export
import pytorch_lightning as lit
import torch
from torch import nn, optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from transformers import AutoModel

default_model_name = "distilbert-base-uncased-finetuned-sst-2-english"


class LightningReapp(lit.LightningModule):
    def __init__(self, config={}, pretrained_model_name=default_model_name):
        super().__init__()

        self.lr = config.get('lr', 1e-4)
        self.hidden_layer_size = config.get('hidden_layer_size', 50)
        self.save_hyperparameters()

        # Initialize a pretrained model
        self.bert = AutoModel.from_pretrained(pretrained_model_name)

        # Turn off autograd for bert encoder
        for param in self.bert.parameters():
            param.requires_grad = False

        self.classifier = nn.Sequential(
            nn.Linear(768, self.hidden_layer_size),
            nn.ReLU(),
            nn.Linear(self.hidden_layer_size, 7),
            nn.ReLU(),
        )

        # define metrics
        self.train_loss = lit.metrics.MeanSquaredError()
        self.val_loss = lit.metrics.MeanSquaredError()
        self.r2score = lit.metrics.R2Score()
        self.explained_var = lit.metrics.ExplainedVariance()

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids, attention_mask)
        last_hidden_state = output.last_hidden_state
        avg = get_avg_masked_encoding(last_hidden_state, attention_mask)
        out = self.classifier(avg).squeeze()
        return out


    def configure_optimizers(self):
        """
        Return whatever optimizers and learning rate schedulers you want here.
        At least one optimizer is required.
        """
        optimizer = optim.AdamW(self.parameters(), lr=self.lr)
        lr_scheduler = {
            'optimizer': optimizer,
            'scheduler': ReduceLROnPlateau(optimizer, mode='min'),
                        'name': 'lr',
                        "monitor": "val_loss",
                        }
        return lr_scheduler

    def training_step(self, batch, batch_idx):
        # destructure batch
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        score = batch["score"]
        # Compute the loss
        output = self(input_ids, attention_mask)
        loss = self.train_loss(output.sum(dim=1), score)
        self.log("train_loss", loss, on_step=True, on_epoch=True)
        return loss

    # VALIDATION LOOP
    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        expected = batch["score"]
        output = self(input_ids, attention_mask)
        observed = output.sum(dim=1)
        outputs = {
            "val_loss": self.val_loss(observed, expected),
            'lr': self.lr,
            'r2score': self.r2score(observed, expected),
            'explained_var': self.explained_var(observed, expected)
        }
        self.log_dict(outputs)
        return outputs

    def validation_epoch_end(self, outputs):
        val_loss = self.val_loss.compute()
        train_loss = self.train_loss.compute()
        loss_distance = torch.abs(val_loss - train_loss)
        self.log("loss_distance", loss_distance)
        self.val_loss.reset()




def get_avg_masked_encoding(state: torch.Tensor, attention_mask: torch.Tensor):
    """[summary]
    For B = batch size, L = encoding length, F = feature vector:
    Args:
        state (torch.Tensor): (B, L, F)
        attention_mask (torch.Tensor): (B, L)
    Returns:
        torch.Tensor: (B, F), where L has been masked by `attention_mask` and then averaged.
            Each vector in the batch represents the average feature vector for the masked_encoding
    """
    encodings = state.unbind(dim=0)  # split the batch up
    new_tensors = []
    # TODO: VECTORIZE
    for i in range(len(encodings)):
        # for each element in the encoding dimension, get the elements that aren't padding using the attention_mask
        encoding_mask = attention_mask[i]
        # Find the indices where attention_mask > 0 (where the actual tokens are without padding)
        indices = encoding_mask.nonzero(as_tuple=True)[0]
        masked_encoding = (
            encodings[i].index_select(0, indices).squeeze()
        )  # torch.Size([len(indices), F])
        avg_feature = torch.mean(masked_encoding, dim=0)
        new_tensors.append(avg_feature)

    return torch.stack(new_tensors)